In [1]:
import os
import numpy as np
import random as rn

import tensorflow as tf
from keras import backend as k

SEED = 29082013
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

Using TensorFlow backend.


### Load data

In [ ]:

dataset = pd.read_csv(
    'AAPL_2006-01-01_to_2018-01-01.csv', 
    parse_dates=['Date']
).set_index('Date')
 
dataset.head()

In [ ]:
dataset.dtypes

In [ ]:
dataset.index

In [ ]:
dataset.index.min(), dataset.index.max(), dataset.shape

In [ ]:
set_entrenamiento = dataset['2010':'2016'].iloc[:, 1:2]
set_validacion = dataset['2017'].iloc[:, 1:2]

print(set_entrenamiento.index.min(), set_entrenamiento.index.max(), set_entrenamiento.shape)
print(set_validacion.index.min(), set_validacion.index.max(), set_validacion.shape)

In [ ]:
set_validacion.shape, set_entrenamiento.shape

In [ ]:
display(set_entrenamiento.head(2))
display(set_entrenamiento.tail(2))
display(set_validacion.head(2))
display(set_validacion.tail(2))

In [ ]:
set_entrenamiento['High'].plot(legend=True)
set_validacion['High'].plot(legend=True)
plt.legend(['Entrenamiento (2006-2016)', 'Validación (2017)'])
plt.show()

In [ ]:
time_step = 60
set_entrenamiento[-1*time_step:].shape

In [ ]:
real_test = set_entrenamiento[-1*time_step:]

display(real_test.head(2))
display(real_test.tail(2))

## Modeling

In [ ]:
# Normalización del set de entrenamiento

sc = MinMaxScaler(
    feature_range=(0, 1)
)
#set_entrenamiento_escalado = sc.fit_transform(set_entrenamiento)
set_entrenamiento_escalado = set_entrenamiento.copy().values
set_entrenamiento_escalado.shape

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
X_train = []
Y_train = []
m = len(set_entrenamiento_escalado)

for i in range(time_step, m):
    X_train.append(
        set_entrenamiento_escalado[i - time_step: i, 0]
    )
    Y_train.append(
        set_entrenamiento_escalado[i, 0]
    )
    
X_train, Y_train = np.array(X_train), np.array(Y_train)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
display(X_train[0])

In [ ]:
X_train = np.reshape(
    X_train, 
    (X_train.shape[0], X_train.shape[1], 1)
)

display(X_train[0])

In [ ]:
# Red LSTM
#
dim_entrada = (X_train.shape[1], 1)
dim_salida = 1
na = 50

In [ ]:
modelo = Sequential()
modelo.add(
    LSTM(units=na, input_shape=dim_entrada)
)
modelo.add(
    Dense(units=dim_salida)
)
modelo.compile(
    optimizer='rmsprop', 
    loss='mse'
)
modelo.fit(
    X_train, 
    Y_train, 
    epochs=10, 
    batch_size=32
)

In [ ]:
set_validacion.shape, real_test.shape

In [ ]:
#
# Validación (predicción del valor de las acciones)
#
x_test = real_test.values
#x_test = sc.transform(x_test)
x_test.shape

In [ ]:
x_test.shape, x_test[-1*time_step:].shape

In [ ]:
x_test.shape

In [ ]:
nn = set_validacion.shape[0]
index_validacion = set_validacion.index.copy()
_x_test = x_test.copy()
list_forecast = []

for i in range(nn):
    test_predict = np.array(
        [_x_test[-1*time_step:]]
    )
    
    test_predict = np.reshape(
        test_predict, (test_predict.shape[0], test_predict.shape[1], 1)
    )

    prediccion = modelo.predict(test_predict)

    _x_test = np.append(_x_test, prediccion, axis=0)

    #list_forecast.append(sc.inverse_transform(prediccion)[0][0])
    list_forecast.append(prediccion[0][0])

In [ ]:
x_test.shape, _x_test.shape, len(list_forecast)

In [ ]:
serie_result = pd.Series(list_forecast, index=index_validacion)
serie_result.head(10)

In [ ]:
set_validacion['High'].head(10)

In [ ]:
# Graficar resultados
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(25, 8))

axes.plot(set_entrenamiento['High'], color='green', label='Valor real pasado')
plt.plot(set_validacion['High'], color='red', label='Valor real de la métrica')
plt.plot(serie_result, color='blue', label='Predicción de la métrica')


#axes.axvline(set_entrenamiento.index[1698], color="blue", linestyle="--", lw=2)
#axes.axvline(set_entrenamiento.index[2300], color="blue", linestyle="--", lw=2)

#axes.axvline(set_validacion_init.index[160], color="blue", linestyle="--", lw=2)
#axes.axvline(set_validacion_init.index[230], color="blue", linestyle="--", lw=2)
#axes.axvline(set_validacion_init.index[240], color="blue", linestyle="--", lw=2)

#plt.ylim(1.1 * np.min(prediccion)/2, 1.1 * np.max(prediccion))

plt.xlabel('Tiempo')
plt.ylabel('Métrica')
plt.legend()
plt.show()